In [1]:
import pandas
pandas.__version__

'1.2.4'

In [2]:
import pandas as pd
import sqlite3 as db

In [3]:
# Read the data from the sheet named "data"
df = pd.read_excel('skill_test_data.xlsx', sheet_name='data')

In [4]:
# Create the pivot table
pivot_table = pd.pivot_table(df, 
                             values=['Spend', 'Attributed Rev (1d)', 'Imprs', 'Visits', 'New Visits', 'Transactions (1d)','Email Signups (1d)'], 
                             index=['Platform (Northbeam)'], 
                             aggfunc='sum', 
                             margins='true',
                             margins_name='Grand Total')

In [5]:
# Rearrange the Columns to match the pivot_table Columns from Excel
pivot_table = pivot_table.reindex(columns=['Spend', 'Attributed Rev (1d)', 'Imprs', 'Visits', 'New Visits', 'Transactions (1d)','Email Signups (1d)'])

# Sort the pivot table by revenue in descending order
pivot_table = pivot_table.sort_values(by='Attributed Rev (1d)', ascending=False)

# Round the values to two decimal places
pivot_table = pivot_table.round(2)

In [15]:
print(pivot_table)

                         Spend  Attributed Rev (1d)      Imprs   Visits  \
Platform (Northbeam)                                                      
Grand Total           59409.97            117093.57  2761059.0  29193.0   
Google Ads            28228.10             90448.41  1476413.0  16391.0   
Facebook Ads          30073.42             49872.23  1225850.0  18298.0   
Organic                   0.00             45056.68        0.0  24377.0   
Organic Search            0.00             24479.03        0.0   5575.0   
Unattributed              0.00              4860.46        0.0      0.0   
Instagram Organic         0.00              2614.29        0.0    557.0   
Other                     0.00              2205.55        0.0    643.0   
Other Email               0.00              1521.82        0.0    435.0   
TikTok                 1108.45              1360.64    58796.0    526.0   
Facebook Organic          0.00              1231.29        0.0    544.0   
YouTube Organic          

In [7]:
# Connect to the SQLite database
conn = db.connect('data_test.db')

In [8]:
# Create a table in the database
create_sql = "CREATE TABLE IF NOT EXISTS pivot_table ([Platform (Northbeam)] VARCHAR, Spend REAL, [Attributed Rev (1d)] REAL, Imprs REAL, Visits INTEGER, [New Visits] INTEGER, [Transactions (1d)] REAL, [Email Signups (1d)] REAL)"
cursor = conn.cursor()
cursor.execute(create_sql)

In [11]:
# Insert the values into the database
for row in pivot_table.itertuples():
    insert_sql = f"INSERT INTO pivot_table ([Platform (Northbeam)], Spend, `Attributed Rev (1d)`, Imprs, Visits, `New Visits`, `Transactions (1d)`, `Email Signups (1d)`) VALUES ('{row[0]}', {row[1]}, {row[2]}, {row[3]}, {row[4]}, {row[5]}, {row[6]}, {row[7]})"
    cursor.execute(insert_sql)

In [12]:
conn.commit()